<a href="https://colab.research.google.com/github/ZeinabAmiri/TravelTideSegmentation/blob/main/Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving CSV Metrics.csv to CSV Metrics.csv


In [4]:
import pandas as pd
import io

csv_content = uploaded['CSV Metrics.csv']

data = pd.read_csv(io.BytesIO(csv_content))


Q1_page_clicks = data['page_clicks'].quantile(0.25)
Q3_page_clicks = data['page_clicks'].quantile(0.75)
IQR_page_clicks = Q3_page_clicks - Q1_page_clicks

Q1_base_fare = data['base_fare_usd'].quantile(0.25)
Q3_base_fare = data['base_fare_usd'].quantile(0.75)
IQR_base_fare = Q3_base_fare - Q1_base_fare


lower_bound_page_clicks = Q1_page_clicks - 1.5 * IQR_page_clicks
upper_bound_page_clicks = Q3_page_clicks + 1.5 * IQR_page_clicks

lower_bound_base_fare = Q1_base_fare - 1.5 * IQR_base_fare
upper_bound_base_fare = Q3_base_fare + 1.5 * IQR_base_fare


outliers_page_clicks = data[(data['page_clicks'] < lower_bound_page_clicks) | (data['page_clicks'] > upper_bound_page_clicks)]
outliers_base_fare = data[(data['base_fare_usd'] < lower_bound_base_fare) | (data['base_fare_usd'] > upper_bound_base_fare)]
print(outliers_page_clicks)
print(outliers_base_fare)


# Remove outliers from the data (replace 'outliers_page_clicks' and 'outliers_base_fare' with your cleaned data)
cleaned_data = data[~data['user_id'].isin(outliers_page_clicks['user_id'])]
cleaned_data = cleaned_data[~cleaned_data['user_id'].isin(outliers_base_fare['user_id'])]

# Save the cleaned data to a CSV file (replace 'cleaned_data.csv' with your desired file name)
cleaned_data.to_csv('cleaned_data.csv', index=False)



# Calculate the "number of flights with a discount for each user"
discount_flight_count = cleaned_data.groupby('user_id')['flight_discount_amount'].apply(lambda x: (x > 0).sum()).reset_index()

# Calculate the total number of flights for each user
total_flight_count = cleaned_data.groupby('user_id')['flight_discount_amount'].count().reset_index()

# Merge the two DataFrames
flight_discount_data = pd.merge(total_flight_count, discount_flight_count, on='user_id', suffixes=('_total', '_discount'))

# Calculate the discount_flight_proportion
flight_discount_data['discount_flight_proportion'] = (flight_discount_data['flight_discount_amount_discount'] / flight_discount_data['flight_discount_amount_total']) * 100



# Calculate the "average flight discount for each user"
average_flight_discount = cleaned_data.groupby('user_id')['flight_discount_amount'].mean().reset_index()


import numpy as np

def calculate_distance(row):
    """
    Calculate the distance between two sets of geographical coordinates (latitude and longitude) using the Haversine formula.
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Extract coordinates from the row
    lat1 = np.radians(row['home_airport_lat'])
    lon1 = np.radians(row['home_airport_lon'])
    lat2 = np.radians(row['destination_airport_lat'])
    lon2 = np.radians(row['destination_airport_lon'])

    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance_km = R * c

    return distance_km





# Make sure you replace these column names with the actual column names in your DataFrame
cleaned_data['home_airport_lat'] = ...  # Replace with the appropriate column
cleaned_data['home_airport_lon'] = ...  # Replace with the appropriate column
cleaned_data['destination_airport_lat'] = ...  # Replace with the appropriate column
cleaned_data['destination_airport_lon'] = ...  # Replace with the appropriate column

# Calculate the distance using the Haversine formula
cleaned_data['distance_km'] = cleaned_data.apply(calculate_distance, axis=1)

# Calculate the scaled_ADS_per_km
cleaned_data['scaled_ADS_per_km'] = cleaned_data['flight_discount_amount'] / cleaned_data['distance_km']



       user_id sign_up_date   birthdate gender  married  has_children  \
7          483   2021-04-18  1981-09-21      F     True          True   
20       11337   2021-06-24  1984-09-07      M    False         False   
31       22411   2021-07-20  1965-11-06      F    False         False   
32       22612   2021-07-20  1978-07-03      M    False         False   
56       20071   2021-07-15  1973-06-09      M    False         False   
...        ...          ...         ...    ...      ...           ...   
49897   328031   2022-09-18  1986-11-17      F     True         False   
49914   357465   2022-10-10  1978-07-28      F    False         False   
49958   402769   2022-11-09  1985-10-01      M    False         False   
49984   411648   2022-11-14  1991-11-03      F    False         False   
49987   507202   2023-01-03  1985-02-03      M    False         False   

      home_country   home_city home_airport  home_airport_lat  \
7              usa   san diego          SAN            32.

TypeError: ignored